In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("churn-bigml-80.csv")

In [5]:
df["Churn_int"] = df["Churn"].astype(int)

In [7]:
variables_utiles = [
    'Account length', 'Total day minutes', 'Total eve minutes', 
    'Total night minutes','Total intl charge', 'Customer service calls',
    'International plan', 'Churn_int'
]

In [9]:
df_modelo = df[variables_utiles]

In [11]:
df_modelo = pd.get_dummies(df_modelo, columns=["International plan"], drop_first = True)

In [13]:
df_modelo.dtypes

Account length              int64
Total day minutes         float64
Total eve minutes         float64
Total night minutes       float64
Total intl charge         float64
Customer service calls      int64
Churn_int                   int32
International plan_Yes       bool
dtype: object

In [15]:
# To salvando o data frame só com as variaveis que vou usar nos Modelos
df_modelo.to_csv("df_modelo_1.csv", index=False)

# ML

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [19]:
# Separando variaveis treino e objetivo
X = df_modelo.drop("Churn_int", axis=1)
y = df_modelo["Churn_int"]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 777, stratify = y)

In [23]:
# Padronizacao dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [25]:
# Definicao do Modelo MLP Classifier
modelo_mlp = MLPClassifier(
    hidden_layer_sizes = (100,100,50), # Define o numero de neuronios nas camada, aqui sao 3vcamadas: 100, 100 e 50 neuronios
    activation = "relu", # funcao de ativacao, outras(tanh, logistic)
    solver="adam", # otimizador
    alpha=0.01,
    max_iter=500, # numero de maximo de iteracoes
    random_state=777
)

# Treinamento
modelo_mlp.fit(X_train_scaled, y_train)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(100, 100, 50), max_iter=500,
              random_state=777)

In [27]:
# Prediccao

y_predi_neural = modelo_mlp.predict(X_test_scaled)

In [29]:
print(confusion_matrix(y_test, y_predi_neural))

[[423  33]
 [ 23  55]]


In [31]:
print(classification_report(y_test, y_predi_neural, digits=4))

              precision    recall  f1-score   support

           0     0.9484    0.9276    0.9379       456
           1     0.6250    0.7051    0.6627        78

    accuracy                         0.8951       534
   macro avg     0.7867    0.8164    0.8003       534
weighted avg     0.9012    0.8951    0.8977       534



# testando com csv balanceado

In [33]:
df_balanceado = pd.read_csv("df_balanceado.csv")

In [40]:
# definindo as variaveis teste e objetivo
X_b = df_balanceado.drop("Churn_int", axis=1)
y_b = df_balanceado["Churn_int"]

In [42]:
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size = 0.2, random_state = 777, stratify = y_b)

In [44]:
# Padronizacao dos dados
scaler = StandardScaler()
X_train_scaled_b = scaler.fit_transform(X_train_b)
X_test_scaled_b = scaler.fit_transform(X_test_b)

In [46]:
# treinando o modelo com o novo arquivo balanceado
modelo_mlp.fit(X_train_scaled_b, y_train_b)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(100, 100, 50), max_iter=500,
              random_state=777)

In [48]:
y_predi_b = modelo_mlp.predict(X_test_scaled_b)

In [50]:
print(confusion_matrix(y_test_b, y_predi_b))
print(classification_report(y_test_b, y_predi_b, digits=4))

[[63 15]
 [25 53]]
              precision    recall  f1-score   support

           0     0.7159    0.8077    0.7590        78
           1     0.7794    0.6795    0.7260        78

    accuracy                         0.7436       156
   macro avg     0.7477    0.7436    0.7425       156
weighted avg     0.7477    0.7436    0.7425       156



## obs: Modelo neuronal MLP Classifier é pior que random forest nesse caso:

    - La red neuronal falló más veces al identificar correctamente a los clientes que realmente se iban.
    - Random Forest acertó más, incluso cuando equilibramos la cantidad de clientes que cancelan y los que no.

## Qué aprendimos?

Aunque la red neuronal es una técnica poderosa, no siempre es la mejor opción, especialmente si los datos no tienen el volumen o el formato ideal para ella.
En este caso, Random Forest fue más confiable, más fácil de interpretar y ofreció mejores resultados.

💡 Conclusión final: No siempre el modelo más complejo es el mejor. Lo importante es probar, comparar y entender cuál se adapta mejor a tu problema y tus datos.

## Porque no usar Rede Neural en este caso: 

1. Pocos datos para un modelo tan complejo
Las redes neuronales necesitan muchos dados variados para aprender patrones complejos. En este caso, usamos apenas unas 700–800 filas después de balancear los datos. Para una red con cientos de neuronas ocultas, eso es muy poco.
📉 Resultado: el modelo no aprende bien y se confunde.

2. Los datos no tienen suficiente complejidad
Random Forest funciona muy bien con datos tabulares (en forma de tabla), como los que tenemos aquí. Las redes neuronales brillan más cuando hay imágenes, audio o texto, o cuando los datos tienen relaciones no lineales muy complejas.

🧱 Aquí los datos son “sencillos” (números de llamadas, minutos hablados, si tiene plan internacional o no...), y Random Forest puede entenderlos mejor.

3. Escala y normalización importan mucho
Las redes neuronales son muy sensibles a los valores numéricos. Si una variable tiene números grandes y otra pequeños, la red puede enfocarse en la equivocada. Aunque usamos StandardScaler, la calidad de la estandarización depende mucho del conjunto de datos original.

Random Forest no tiene ese problema: no le importa si los números son grandes o pequeños.

# Testando modelo com el archivo 20.csv (Simulando la realidad con nuevos datos) 

In [52]:
# Lendo os dados csv 20
df_test_real = pd.read_csv("churn-bigml-20.csv")

In [54]:
# Criando uma nova coluna de churn como numero interiro 
df_test_real["Churn_int"] = df_test_real["Churn"].astype(int)

In [56]:
# Criando um data frame só com as variables uteis
df_test_real = df_test_real[variables_utiles]

In [58]:
# Transformando a variavel categorica em booleana
df_test_real = pd.get_dummies(df_test_real, columns=["International plan"], drop_first=True)

In [60]:
# Salvado arquivo listo para ser usado nos modelos reais de teste
df_test_real.to_csv("df_20%_tratado.csv", index=False)

In [62]:
# separando variaveis
treino = df_test_real.drop("Churn_int", axis=1)
objetivo = df_test_real["Churn_int"]

In [69]:
# treino e teste
X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(treino, objetivo, test_size = 0.2, random_state = 777, stratify = objetivo)

In [71]:
# treinar o modelo
modelo_mlp.fit(X_train_real, y_train_real)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(100, 100, 50), max_iter=500,
              random_state=777)

In [73]:
# modelo tentando predizer
y_pred_real = modelo_mlp.predict(X_test_real)

In [75]:
print(confusion_matrix(y_test_real, y_pred_real))

[[115   0]
 [ 19   0]]


In [77]:
print(classification_report(y_test_real, y_pred_real, digits=4))

              precision    recall  f1-score   support

           0     0.8582    1.0000    0.9237       115
           1     0.0000    0.0000    0.0000        19

    accuracy                         0.8582       134
   macro avg     0.4291    0.5000    0.4618       134
weighted avg     0.7365    0.8582    0.7927       134



C:\Users\jfaze\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jfaze\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\jfaze\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
